In [1]:
import numpy as np
from itertools import combinations
from itertools import chain

In [2]:
def signPermutation(p):
 # explicitly using cycle notation of permutation
 # input: p is a list of permutation indices
 # i.e. [x_sorted, p] = sort(x_unsorted)

        n = len(p)
        visited = [False]*n
        sign = 1
        cycle = []
        for k in range(n):
                if (not visited[k]):
                        ct = k
                        L = 0
                        tempcyc = []
                        while (not visited[ct]):
                                L += 1
                                visited[ct] = True
                                ct = p[ct]
                                tempcyc.append(ct)
                        if len(tempcyc) > 1:
                            cycle.append(tempcyc)
                        if L%2 == 0:
                                sign = -1*sign
        return sign, cycle
    
def hextobin(ini_string):
        return bin(int(ini_string,16))[2:]

In [3]:
# Routine that writes the anstisymmetrizer for any combination
# A(pqrst...):

def antisymmetrizer(string_in):
    from itertools import permutations
    
    L = string_in.split('/')

    ct = 1
    P = list(L[0])
    ASYM = ''
    if len(L) == 2:
        # We do not allow for permutations among indices in each group
        # on either side of the slash
        permlistQ = list(permutations(range(len(L[1]))))
    else:
        # find all permutations of the indices in the group
        sign = []
        cycle = []
        permlistP = []
        strcyc = []
        for p in list(permutations(range(len(L[0])))):
            idx = np.argsort(np.asarray(p))
            s,c = signPermutation(idx)
            temp = ''
            for i in range(len(p)):
                temp += P[p[i]]
                temp2 = ''
                for j in range(len(c)):
                    for k in range(len(c[j])-1):   
                        temp2 += '('+P[p[c[j][0]]]+P[p[c[j][len(c[j])-k-1]]]+')'
            sign.append(c)
            cycle.append(c)
            strcyc.append(temp2)
            permlistP.append(temp)
            
            if s > 0:
                if strcyc[-1] == '':
                    ASYM += '+(1)'
                else:
                    ASYM += '+'+strcyc[-1]
            else:
                ASYM += '-'+strcyc[-1]
                
        print('{}'.format(ASYM))
            
            

    return permlistP,sign,cycle,strcyc


In [20]:
class XOperator:
    
    def __init__(self,dag,index,spin):
        
        self.dag = dag # -1 if annihilator, 1 if creator
        self.spin = spin # '1' if alpha, '0' if beta
        self.index = index # string p, q, r, s, i, j, k, l, a, b, c, d, etc.
        if self.index == 'i' or self.index == 'j' or self.index == 'k' or self.index == 'l':
            self.character = 1 # fixed hole
        elif self.index == 'm' or self.index == 'n' or self.index == 'o' or self.index == 'v':
            self.character = 2 # free hole
        elif self.index == 'a' or self.index == 'b' or self.index == 'c' or self.index == 'd':
            self.character = 3 # fixed particle
        elif self.index == 'e' or self.index == 'f' or self.index == 'g' or self.index == 'h':
            self.character = 4 # free particle
        elif self.index == 'p' or self.index == 'q' or self.index == 'r' or self.index == 's':
            self.character = 5 # generic index
            
        if self.character == 1 or self.character == 2:
            self.Yopdag = -1*self.dag
        elif self.character == 3 or self.character ==4:
            self.Yopdag = self.dag
        else:
            self.Yopdag = 2

class Expression:
    
    def __init__(self,expr_label,sign,weight,matrix_element,operator_list): 
        self.sign = sign
        self.weight = weight
        self.matrix_element = matrix_element # contracted & uncontracted indices
        self.operator_list = operator_list # list of FermiOperator objects
        self.expr_label = expr_label
    
    def is_normal_fermi(self):
        FLAG = True
        N = len(self.operator_list)
        for i in range(N-1):
            X1 = self.operator_list[N-1-i]
            X2 = self.operator_list[N-2-i]
            if X1.ph_character == 2 or X2.ph_character == 2:
                return 'Cannot determine Fermi normal ordering with generic indices'
            else:
                Y1 = YOperator(X1)
                Y2 = YOperator(X2)
                if Y1.dag == 1 and Y2.dag == -1:
                    FLAG = False
        return FLAG
    
    def print_expression(self):
        
        printObj = []
        
        if self.sign == 1:
            printObj += ['+']
        if self.sign == -1:
            printObj += ['-']
            
            
        printObj += ['weight = '+str(self.weight)]
        
        printObj += ['matel = '+str(self.expr_label)+'('+str(self.matrix_element)+')']
        
        temp = []
        for i in range(len(self.operator_list)):
            if self.operator_list[i].spin == 1:
                if self.operator_list[i].dag == 1:
                    temp += [self.operator_list[i].index+'+']
                if self.operator_list[i].dag == -1:
                    temp += [self.operator_list[i].index]
            if self.operator_list[i].spin == 0:
                if self.operator_list[i].dag == 1:
                    temp += ['~'+self.operator_list[i].index+'+']
                if self.operator_list[i].dag == -1:
                    temp += ['~'+self.operator_list[i].index]
                
        printObj += temp     
        
        print(printObj)
        
def contraction_check(list_contr):
    # Would be good to make a quick parser method to check whether a list of full contractions
    # is allowable or not
        return bool

def contraction_fermi(X1,X2):
    # will return a delta function ONLY d(pq) we won't contract terms
    # that give 0 anyway! return '0' is a flag that the contraction doesn't
    # work. In practice, we should try to never perform contractions that are 0
    
    if X1.spin == X2.spin:
    
        # h-h case X^dag X
        # (i,m), (i,p), (m,n)
        if X1.character == 1 or X1.character == 2 or X1.character == 5 and \
           X2.character == 1 or X2.character == 2 or X2.character == 5:
            
            if X1.character == 5 and X2.character == 5: # should really never contract p with q
                return '0'
            elif X1.character == 1 and X2.character == 1: # contraction of two fixed indices
                return '0'
            elif X1.dag == 1 and X2.dag == -1:
                return [X1.index, X2.index]
            else:
                return '0'

        # p-p case X X^dag
        # (a,e), (a,p), (e,f)
        elif X1.character == 3 or X1.character == 4 or X1.character == 5 and \
             X2.character == 3 or X2.character == 4 or X2.character == 5:
            
            if X1.character == 5 and X2.character == 5: # should really never contract p with q
                return '0'
            elif X1.character == 3 and X2.character == 3: # contraction of two fixed indices
                return '0'
            elif X1.dag == -1 and X2.dag == 1:
                return [X1.index, X2.index]
            else:
                return '0'

        else:
            return '0'
    else:
        return '0'
        
    

class VEV_Expression:         
     
        def __init__(self, list_of_expressions):
            
            self.operator_list_labels = [list_of_expressions[i].expr_label for i in range(len(list_of_expressions))]
            
            # list of the operator lists of each expression
            self.operator_list = [list_of_expressions[i].operator_list for i in range(len(list_of_expressions))]
            
            # flattened list of the operators of each expression
            self.operator_list_flat = [list_of_expressions[i].operator_list[j] for i in range(len(list_of_expressions)) \
                                      for j in range(len(list_of_expressions[i].operator_list))]
            
            # number of expressions passed in
            self.num_expr = len(list_of_expressions)
            
            self.num_proj_lines = len(list_of_expressions[0].operator_list)
            self.num_tot_lines = len(self.operator_list_flat)
            
            # list containing the association of each operator in operator_list_flat to what number expression passed in
            self.operator_list_exp = []
            for i in range(len(list_of_expressions)):
                for j in range(len(list_of_expressions[i].operator_list)):
                    self.operator_list_exp += [i]
            
            self.operator_expr_idx = []
            for i in range(len(list_of_expressions)):
                self.operator_expr_idx.append([j for j in range(len(self.operator_list_flat)) if \
                                              self.operator_list_exp[j] == i])
            
            # flattened list of indices for each operator
            self.indices = [self.operator_list_flat[i].index for i in range(len(self.operator_list_flat))]
            
            # flattened list of symmetry label (i.e. spin + abelian irrep) for each operator
            self.sym = [self.operator_list_flat[i].spin for i in range(len(self.operator_list_flat))]
            
            # flattened list of character (i.e. fixed/free particle/hole/generic) for each operator
            self.characters = [self.operator_list_flat[i].character for i in range(len(self.operator_list_flat))]
            
            # number of symmetry representations, including spin (= 2*number of abelian irreps for spatial symmetry)
            self.nrep = len(list(set(self.sym)))
            
            # sign, weight, and matrix element
            self.sign = 1
            self.weight = 1
            self.matrix_element = []
            self.expr_labels = []
            for E in list_of_expressions:
                self.sign *= E.sign
                self.weight *= E.weight
                self.matrix_element.append(E.matrix_element)
                self.expr_labels.append(E.expr_label)
                
        def organize_indices(self):
            # generate a list where each element is a list of indices (numerical index) of operators in VEV
            # that have the same spin/symmetry label and hole/particle character (generic indices always included)
            # Returns
            # idx_hole_list:
            # e.g. [[i], [p,r], [m, o]] <- hole_alpha (numerical indices)
            
            idx_hole_list = []
            idx_particle_list = []
            idx_hole_list_2 = []
            idx_particle_list_2 = []
            
            for i in range(self.nrep):
                if self.nrep > 1:
                    temp = [j for j in range(len(self.indices)) if i == self.sym[j]]
                else:
                    temp = list(range(len(self.indices)))
                
                tempx = [temp[j] for j in range(len(temp)) if self.characters[temp[j]]==2]
                tempy = [temp[j] for j in range(len(temp)) if self.characters[temp[j]]==4]
                
                temp2 = []
                temp3 = []
                
                temp4 = []
                temp5 = []
                for k in range(self.num_expr):
                    temp6 = [tempx[v] for v in range(len(tempx)) if self.operator_list_exp[tempx[v]] == k]
                    temp7 = [tempy[v] for v in range(len(tempy)) if self.operator_list_exp[tempy[v]] == k]
                    if len(temp6) > 0:
                        temp4.append(temp6)
                    if len(temp7) > 0:
                        temp5.append(temp7)
                
                
               # temp2.append([temp[j] for j in range(len(temp)) if self.characters[temp[j]] == 1])
                temp2.append([temp[j] for j in range(len(temp)) if self.characters[temp[j]] == 5])
                
                idx_hole_list_2.append(temp2+temp4)
                idx_particle_list_2.append(temp2+temp5)
                
                temp2.append([temp[j] for j in range(len(temp)) if self.characters[temp[j]] == 2])
                
               # temp3.append([temp[j] for j in range(len(temp)) if self.characters[temp[j]] == 3])
                temp3.append([temp[j] for j in range(len(temp)) if self.characters[temp[j]] == 5])
                temp3.append([temp[j] for j in range(len(temp)) if self.characters[temp[j]] == 4])
    
                idx_hole_list.append(temp2)
                idx_particle_list.append(temp3)
    
            return idx_hole_list, idx_particle_list, idx_hole_list_2, idx_particle_list_2
        
        
        def contraction_check(self,p1,p2):
            
            # connectedness here is defined as contracting a generic index to a 
            # FREE hole/particle index!
            
            X1 = self.operator_list_flat[p1]
            X2 = self.operator_list_flat[p2]
            if X1.character != 5 and X2.character != 5:
                FLAG = True
                connect = False
            else:
                X1 = self.operator_list_flat[p1]
                X2 = self.operator_list_flat[p2]
                if X1.character == 1 and X2.character == 5:
                    connect = False
                    if X1.dag == 1 and X2.dag == -1:
                        FLAG = True
                    else:
                        FLAG = False
                elif X1.character == 5 and X2.character == 2:
                    connect = True
                    if X1.dag == 1 and X2.dag == -1:
                        FLAG = True
                    else:
                        FLAG = False
                elif X1.character == 3 and X2.character == 5:
                    connect = False
                    if X1.dag == -1 and X2.dag == 1:
                        FLAG = True
                    else:
                        FLAG = False
                elif X1.character ==5 and X2.character == 4:
                    connect = True
                    if X1.dag == -1 and X2.dag == 1:
                        FLAG = True
                    else:
                        FLAG = False
                else:
                    FLAG = False
                    
            return FLAG, connect
                         
        
        def generate_pair_list(self, idx_h_list, idx_p_list):
            
#            idx_h_list, idx_p_list, _, _ = self.organize_indices()
            
            # hole (particle) creators (annhiliators) need generic annhiliators (creators)
            # hole (particle) annihilators (creators) needs generic creators (annhilators)
            
            n_sym_rep = len(idx_h_list) # should equal len(idx_p_list)
            
            ct_pair = 0
        
            pair_list = []
            connect_idx = [[] for i in range(self.num_expr-2)]
            
            for n in range(n_sym_rep):
                LH = idx_h_list[n]
                LP = idx_p_list[n]

                
                for p in range(len(LH)-1):
                    for q in range(p+1,len(LH)):
                        for i in range(len(LH[p])):
                            for j in range(len(LH[q])):
                                allowed, connect = self.contraction_check(LH[p][i],LH[q][j])
                                if allowed:
                                    pair_list += [(LH[p][i], LH[q][j])]
                                    if connect:
                                        K = max([self.operator_list_exp[LH[p][i]], self.operator_list_exp[LH[q][j]]])
                                        connect_idx[K-2] += [ct_pair]
                                        #connect_idx += [ct_pair]
                                    ct_pair += 1
                                        
                for p in range(len(LP)-1):
                    for q in range(p+1,len(LP)):
                        for i in range(len(LP[p])):
                            for j in range(len(LP[q])):
                                allowed, connect = self.contraction_check(LP[p][i],LP[q][j])
                                if allowed:
                                    pair_list += [(LP[p][i], LP[q][j])]
                                    if connect:
                                        K = max([self.operator_list_exp[LP[p][i]], self.operator_list_exp[LP[q][j]]])
                                        connect_idx[K-2] += [ct_pair]
                                        #connect_idx += [ct_pair]
                                    ct_pair += 1

            return pair_list, connect_idx

        
        def generate_depend_matrix(self, pair_list):
            from itertools import chain
            
            # depMat[i] contains indices in pair_list that are not allowed once
            # pair_list[i] is selected as a contraction
            
           # pair_list = self.generate_pair_list()
              
            depMat = []

            for j1 in range(len(pair_list)):
                A = list(chain.from_iterable([pair_list[j1]]))
                temp = []
                for j2 in range(len(pair_list)):
                    B = list(chain.from_iterable([pair_list[j2]]))
                    if any(i in A for i in B):
                        if j1 != j2:
                            temp += [j2]
                depMat.append(temp)
                        
            return depMat
        
        def generate_full_contractions(self, pair_list, connect_idx, depMat):
            from itertools import combinations
            
            FC_list = []
            fc_pairs = []
            
            def common_member(a, b): 
                a_set = set(a) 
                b_set = set(b) 
                if len(a_set.intersection(b_set)) > 0: 
                    return(True)  
                return(False)   
            
            def is_connected(idx, connect_idx):
                if all([common_member(idx, connect_idx[k]) for k in range(self.num_expr-2)]):
                    return True
                else:
                    return False
            
            def is_allowed(idx,depMat):
                from itertools import chain
                b = chain.from_iterable([depMat[i] for i in idx])
                if common_member(idx,b):
                    return(False)
                else:
                    return(True)
            
            #pair_list, connect_idx = self.generate_pair_list()
            #depMat = self.generate_depend_matrix(pair_list)
            #num_contr = int(len(self.operator_list_flat)/2)
            num_contr = int( (self.num_tot_lines-2*self.num_proj_lines)/2 )
            
            idx = list(combinations(range(len(pair_list)),num_contr))
            
            for i in range(len(idx)):
                #print(i)
                if is_allowed(idx[i],depMat) and is_connected(idx[i],connect_idx):
                    FC_list.append(idx[i])
                    fc_pairs.append([pair_list[idx[i][j]] for j in range(len(idx[i]))])
            
            return FC_list, fc_pairs
        
        
        def collect_distinct_diagrams(self, fc_pairs, idx_hole_list, idx_particle_list):

            def common_member(a, b): 
                a_set = set(a) 
                b_set = set(b) 
                if len(a_set.intersection(b_set)) > 0: 
                    return(True)  
                return(False)  

            def num_common_elements(list1,list2):
                list1_as_set = set(list1)
                intersection = list1_as_set.intersection(list2)
                intersection_as_list = list(intersection)
                return len(intersection_as_list)

            FC_list_R = []
            fc_pairs_R = []
            CT = []
            mult = []
            
            num_symmetries = self.nrep
            num_goldstone = len(fc_pairs)
            num_right_ops = self.num_expr - 2

            for i in range(num_goldstone): # Loop over all Goldstone diagrams

                ct = [[ [0,0] for i1 in range(num_right_ops)] for i2 in range(num_symmetries)]

                IDX = list(chain.from_iterable(fc_pairs[i]))[1::2]
                #print(IDX)

                #temp = [None]*num_symmetries
                
                for sym in range(num_symmetries): # loop over symmetries

                    #temp_hole = [0] * (len(idx_hole_list[j])-1) # size of expressions minus V or F
                    #temp_part = [0] * (len(idx_particle_list[j])-1) # size of expressions minus V or F
                    #temp = [0, 0]

                    for expr in range(len(idx_hole_list[sym])-1): # loop over expressions
                        #temp[k][0] += int(num_common_elements(IDX,idx_hole_list[j][k+1]))
                        #temp[k][1] += int(num_common_elements(IDX,idx_particle_list[j][k+1]))
                        #temp_hole[k] += int(num_common_elements(IDX,idx_hole_list[j][k+1]))
                        #temp_part[k] += int(num_common_elements(IDX,idx_particle_list[j][k+1]))
                        expr_idx = self.operator_list_exp[idx_hole_list[sym][expr+1][0]]-2
                        ct[sym][expr_idx][0] += int(num_common_elements(IDX,idx_hole_list[sym][expr+1]))
                        ct[sym][expr_idx][1] += int(num_common_elements(IDX,idx_particle_list[sym][expr+1]))

                    #temp.append(temp_hole + temp_part)

                #ct.append(temp)
                    
                if i == 0:
                    CT.append(ct)
                    mult.append(1.0)
                    FC_list_R.append(i)
                    fc_pairs_R.append(fc_pairs[i])

                else:
                    flag = False

                    for l in range(len(CT)): # loop over distinct diagrams
                        if list(chain.from_iterable([CT[l]])) == list(chain.from_iterable([ct])):
                            # MUST ALSO CONSIDER ALL PERMUTATIONS OF INDICES WHEN CONSIDERING THE SAME OPERATOR
                            # E.G.
                            # 2 T2 OPERATORS CONTRACTION SCHEMES ARE EQUAL IF WE PERMUTE THE CONTRACTION SCHEMES OF 
                            # INDIVIDUAL T2 OPERATORS
                            
                            mult[l] += 1.0
                            #FC_list_R[l] += [i]
                            #fc_pairs_R[l] += [fc_pairs[i]]
                            flag = True

                    if not flag:
                        CT.append(ct)
                        mult.append(1.0)
                        FC_list_R.append(i)
                        fc_pairs_R.append(fc_pairs[i])
                        
            return CT, mult, FC_list_R, fc_pairs_R
        
        
        def print_full_contractions(self,FC_list,fc_pairs):
        
            for i in range(len(FC_list)):
                temp = []
                for j in range(len(fc_pairs[i])):
                    I = fc_pairs[i][j][0]
                    J = fc_pairs[i][j][1]
                    XI = self.operator_list_flat[I]
                    XJ = self.operator_list_flat[J]
                    strI = XI.index
                    strJ = XJ.index
                    if XI.dag == 1:
                        strI += '+'
                    if XJ.dag == 1:
                        strJ += '+'
                    if XI.spin == 0:
                        strI = '~'+strI
                    if XJ.spin == 0:
                        strJ = '~'+strJ
                    temp += [(strI,strJ)]
                print(temp)
            return
        
        
        def print_diagram_scheme(self,CT,mult,FC_list_R,fc_pairs_R):
            
            num_diagrams = len(CT)
            num_sym = len(CT[0])
            num_right_ops = len(CT[0][0])
            for i in range(num_diagrams):
                print('Diagram {} (multiplicity {}):'.format(i+1, int(mult[i])))
                print('-----------------------------')
                self.print_full_contractions([FC_list_R[i]],[fc_pairs_R[i]])
                for sym in range(num_sym):
                    for expr in range(num_right_ops):
                        print('OPERATOR {} - {}'.format(expr+1, self.operator_list_labels[expr+2]))
                        if CT[i][sym][expr][0] > 0:
                            print('{} hole line(s) of symmetry {}'.format(CT[i][sym][expr][0],sym+1))
                        if CT[i][sym][expr][1] > 0:
                            print('{} particle line(s) of symmetry {}'.format(CT[i][sym][expr][1],sym+1))
                print('\n')
            return
    
        def print_expression(self):
        
            printObj = []

            if self.sign == 1:
                printObj += ['+']
            if self.sign == -1:
                printObj += ['-']


            printObj += ['weight = '+str(self.weight)]
            
            temp3 = ['matel = ']
            for i in range(self.num_expr):
                temp3 += [str(self.expr_labels[i]+'('+self.matrix_element[i]+')')]
                
            printObj += [temp3]

            temp = []
            for i in range(len(self.operator_list)):
                temp2 = []
                for j in range(len(self.operator_list[i])):
                    if self.operator_list[i][j].spin == 1:
                        if self.operator_list[i][j].dag == 1:
                            temp2 += [self.operator_list[i][j].index+'+']
                        if self.operator_list[i][j].dag == -1:
                            temp2 += [self.operator_list[i][j].index]
                    if self.operator_list[i][j].spin == 0:
                        if self.operator_list[i][j].dag == 1:
                            temp2 += ['~'+self.operator_list[i][j].index+'+']
                        if self.operator_list[i][j].dag == -1:
                            temp2 += ['~'+self.operator_list[i][j].index]
                temp.append(temp2)
            printObj.append(temp)    
            
            print(printObj)
            
        def run_main(self):
            
            # organize indices
            idx_hole_list, idx_particle_list, idx_hole_list_2, idx_particle_list_2 = self.organize_indices()
            
            # generate pair list and connection index
            pair_list, connect_idx = self.generate_pair_list(idx_hole_list, idx_particle_list)
            
            # generate dependency matrix
            depMat = self.generate_depend_matrix(pair_list)
            
            # create list of full contractions (Goldstones)
            FC_list, fc_pairs = self.generate_full_contractions(pair_list, connect_idx, depMat)
            
            # consolidate full contractions into distinct Hugenholtz diagrams
            CT, mult, FC_pairs_R, fc_list_R = self.collect_distinct_diagrams(fc_pairs, idx_hole_list_2, idx_particle_list_2)
            
            # print diagrammatic schemes
            self.print_diagram_scheme(CT,mult,FC_pairs_R,fc_list_R)
 

In [21]:
### DO NOT FORGET THE TAYLOR EXPANSION TERM FACTOR IN CC QUANTITIES!!!
## E.G.
## 1/2(V_N * T_2^2)

X1 = XOperator(1,'p',1)
X2 = XOperator(1,'q',1)
X3 = XOperator(-1,'s',1)
X4 = XOperator(-1,'r',1)

V = Expression('V',1,0.25,'pqrs',[X1,X2,X3,X4])

F = Expression('F',1,1,'pq',[X1,X3])

X5 = XOperator(1,'i',1)
X6 = XOperator(1,'j',1)
X7 = XOperator(-1,'b',1)
X8 = XOperator(-1,'a',1)

E2 = Expression('E',1,1,'ijba',[X5,X6,X7,X8])

X9 = XOperator(1,'e',1)
X10 = XOperator(-1,'m',1)

R1 = Expression('R1',1,1,'em',[X9,X10])

X11 = XOperator(1,'f',1)
X12 = XOperator(1,'g',1)
X13 = XOperator(-1,'o',1)
X14 = XOperator(-1,'n',1)

# what's the weight on the T2 operator?
T2 = Expression('T2',1,0.25,'fgno',[X11,X12,X13,X14])

X15 = XOperator(1,'e',1)
X16 = XOperator(1,'h',1)
X17 = XOperator(-1,'v',1)
X18 = XOperator(-1,'m',1)

T22 = Expression('T2',1,0.25,'ehvm',[X15,X16,X17,X18])

X19 = XOperator(1,'i',1)
X20 = XOperator(-1,'a',1)

E1 = Expression('E1',1,1,'ia',[X19,X20])

VEV1 = VEV_Expression([E2,V,T2,T22])
VEV1.print_expression()

['+', 'weight = 0.015625', ['matel = ', 'E(ijba)', 'V(pqrs)', 'T2(fgno)', 'T2(ehvm)'], [['i+', 'j+', 'b', 'a'], ['p+', 'q+', 's', 'r'], ['f+', 'g+', 'o', 'n'], ['e+', 'h+', 'v', 'm']]]


In [22]:
VEV1.run_main()

Diagram 1 (multiplicity 16):
-----------------------------
[('p+', 'o'), ('q+', 'n'), ('s', 'f+'), ('r', 'e+')]
OPERATOR 1 - T2
2 hole line(s) of symmetry 1
1 particle line(s) of symmetry 1
OPERATOR 2 - T2
1 particle line(s) of symmetry 1


Diagram 2 (multiplicity 4):
-----------------------------
[('p+', 'o'), ('q+', 'n'), ('s', 'e+'), ('r', 'h+')]
OPERATOR 1 - T2
2 hole line(s) of symmetry 1
OPERATOR 2 - T2
2 particle line(s) of symmetry 1


Diagram 3 (multiplicity 16):
-----------------------------
[('p+', 'o'), ('q+', 'v'), ('s', 'f+'), ('r', 'g+')]
OPERATOR 1 - T2
1 hole line(s) of symmetry 1
2 particle line(s) of symmetry 1
OPERATOR 2 - T2
1 hole line(s) of symmetry 1


Diagram 4 (multiplicity 64):
-----------------------------
[('p+', 'o'), ('q+', 'v'), ('s', 'f+'), ('r', 'e+')]
OPERATOR 1 - T2
1 hole line(s) of symmetry 1
1 particle line(s) of symmetry 1
OPERATOR 2 - T2
1 hole line(s) of symmetry 1
1 particle line(s) of symmetry 1


Diagram 5 (multiplicity 16):
----------------

In [8]:
iocc = [1,2,3,4,5]
ivir = [5,34,2,4,5]

sum(ivir[:4])

45

In [9]:
sum(ivir)

50

In [17]:
ivir_beta = [i for i in ivir if i%2 == 0]